# Dining philosophers problem

Five silent philosophers sit at a round table with bowls of spaghetti. Forks are placed between each pair of adjacent philosophers.

![](https://upload.wikimedia.org/wikipedia/commons/thumb/7/7b/An_illustration_of_the_dining_philosophers_problem.png/463px-An_illustration_of_the_dining_philosophers_problem.png)

Each philosopher must alternately think and eat. However, a philosopher can only eat spaghetti when they have both left and right forks. Each fork can be held by only one philosopher and so a philosopher can use the fork only if it is not being used by another philosopher. After an individual philosopher finishes eating, they need to put down both forks so that the forks become available to others. A philosopher can take the fork on their right or the one on their left as they become available, but cannot start eating before getting both forks.

Eating is not limited by the remaining amounts of spaghetti or stomach space; an infinite supply and an infinite demand are assumed.

The problem is how to design a discipline of behavior (a concurrent algorithm) such that no philosopher will starve; i.e., each can forever continue to alternate between eating and thinking, assuming that no philosopher can know when others may want to eat or think.

## Compute the transition matrix

Each philospher acts according to the following graph of states:
<img src="./dining_philosophers.png" alt="Drawing" style="width: 70%;"/>

We now want to know, given a set of states like $s = (s_{phil1}, s_{phil2}, s_{phil3}) = (0, 1, 2)$ $\dots$

In [69]:
from pprint import pprint

# Fix probability constants at the top, so we always know where to go to edit things.
h, e = 0.05, 0.75

In [70]:
def compute_successor(qi, qi_prev, qi_foll, h, e):
    def has_lf(q):
        return q in (4, 6, 8, 9)
    def has_rf(q):
        return q in (5, 7, 8, 9)
    def needs_lf(q):
        return q in (2, 5)
    def needs_rf(q):
        return q in (3, 4)
    
    if qi == 0:
        return [([0], 1-h), ([1], h)]
    if qi == 1:
        return [([2], 0.5), ([3], 0.5)]
    if qi == 2:
        if has_rf(qi_prev):
            return [([2], 1)]
        if not has_rf(qi_prev) and not needs_rf(qi_prev):
            return [([4], 1)]
        if not has_rf(qi_prev) and needs_rf(qi_prev):
            return [([2], 1)]
    if qi == 3:
        if has_lf(qi_foll):
            return [([3], 1)]
        else:
            return [([5], 1)]
    if qi == 4:
        if has_lf(qi_foll):
            return [([6], 1)]
        else:
            return [([8], 1)]
    if qi == 5:
        if has_rf(qi_prev):
            return [([7], 1)]
        else:
            return [([8], 1)]
    if qi in (6,7):
        return [([1], 1)]
    
    if qi == 8:
        return [([9], 1)]

    if qi == 9:
        return [([9], e), ([0], 1-e)]

In [71]:
compute_successor(1, 2, 3, 0.05, 0.75)

[([2], 0.5), ([3], 0.5)]

In [79]:
def cartesian_prod(list_to_be_extended, list_to_add):
    """This function computes the cartesian product between two lists made like:
    
    list_to_be_extended = [([a, b], 0.13), ([a, c], 0.24), ([b, c], 0.45)]
    list_to_add = [([x], 0.12), ([y], 0.35)]
    
    where {a, b, c, x, y} are possible states, coming from the graph previously
    presented, and the associated number in the pair is the probability of the
    philosopher to follow/act according to those states.
    
    Please note that `list_to_be_extended` could be an empty list at the beginning
    of the iteration process and, in that case, the `list_to_add` list has to be
    returned.
    """
    
    if not list_to_be_extended:
        return list_to_add
    
    list_with_products_to_return = []
    
    for single_state_to_add, prob_single_state in list_to_add:
        for list_of_states, prob_sequence_states in list_to_be_extended:
            list_with_products_to_return.append((list_of_states + single_state_to_add,
                                                prob_single_state * prob_sequence_states))

    return list_with_products_to_return

    
print("Perform some simple tests:\n")

test_empty_list = []
test_to_add_list = [([3], 0.1), ([4], 0.2)]
print("empty list")
pprint(cartesian_prod(test_empty_list, test_to_add_list))

test_midlen_list = [([0], 0.2), ([1], 0.3)]
print("\n2 elements single-state list")
pprint(cartesian_prod(test_midlen_list, test_to_add_list))

test_long_list = [([0, 0], 0.2), ([0, 1], 0.3)]
print("\n2 elements double-state list")
pprint(cartesian_prod(test_long_list, test_to_add_list))

Perform some simple tests:

empty list
[([3], 0.1), ([4], 0.2)]

2 elements single-state list
[([0, 3], 0.020000000000000004),
 ([1, 3], 0.03),
 ([0, 4], 0.04000000000000001),
 ([1, 4], 0.06)]

2 elements double-state list
[([0, 0, 3], 0.020000000000000004),
 ([0, 1, 3], 0.03),
 ([0, 0, 4], 0.04000000000000001),
 ([0, 1, 4], 0.06)]


In [80]:
input_state = (1, 2, 3)

state_list = []
for ind, _ in enumerate(input_state):
    successor_state = compute_successor(input_state[ind], input_state[ind-1], input_state[(ind+1)%len(input_state)], h, e)
    l = cartesian_prod(state_list, successor_state)
    state_list.extend(l)

pprint(state_list)

[([2], 0.5),
 ([3], 0.5),
 ([2, 4], 0.5),
 ([3, 4], 0.5),
 ([2, 5], 0.5),
 ([3, 5], 0.5),
 ([2, 4, 5], 0.5),
 ([3, 4, 5], 0.5)]
